In [1]:
# from llama_index.indices.postprocessor import SentenceTransformerRerank
# rerank = SentenceTransformer Rerank(top_n=2, model="BAAI/bge-reranker-base")

In [2]:
from scripts import utils
import os
import openai

openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 681b5fd9-c20d-4420-b378-4a7380033d23
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [5]:
# documents[1].text

### Basic RAG Pipeline

In [6]:
from pprint import pprint

In [7]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [8]:
# pprint(document.text)

In [9]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [10]:
query_engine = index.as_query_engine()

In [11]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
pprint(str(response))

('When finding projects to build your experience, there are several steps you '
 'can take. First, you can join existing projects by asking to join someone '
 "else's project if they have an idea. Additionally, you can continue reading, "
 'taking courses, and talking to domain experts to come up with new ideas. '
 'Another approach is to focus on a specific application area where machine '
 'learning has not yet been applied. This can give you the opportunity to '
 'explore unique and creative applications that no one else has done yet. '
 'Finally, you can develop a side hustle or personal project that may or may '
 'not develop into something bigger, as this can stir your creative juices and '
 'strengthen bonds with collaborators.')


### Eval setup using TruLens

In [12]:
eval_questions = []
with open('text/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character 
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [13]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [14]:
pprint(eval_questions)

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter '
 'syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'What is the right AI job for me?']


In [28]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [40]:
from scripts.utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [41]:
import time

with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

        # time.sleep(10)

In [42]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [43]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Sentence Window Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7c07d954de605158c413834559e69273,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_7c07d954de605158c41...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T10:42:44.668484"", ""...",2024-01-10T10:42:55.928617,0.8,1.000,0.30,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 7: A Simple Fram...,[{'args': {'prompt': 'What are the keys to bui...,11,514,0.000788
1,Sentence Window Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3a9f11a10c6c0fc7f2d9a43ebdc928e3,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_3a9f11a10c6c0fc7f2d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T10:42:56.649532"", ""...",2024-01-10T10:43:18.081682,0.9,0.950,0.55,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'prompt': 'How can teamwork contrib...,21,662,0.001045
2,Sentence Window Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_81427322b2fbce0bdb33c59096c4975c,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_81427322b2fbce0bdb3...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T10:43:18.818779"", ""...",2024-01-10T10:43:33.115578,0.8,0.625,0.20,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,[{'args': {'prompt': 'What is the importance o...,14,522,0.000827
3,Sentence Window Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_0d6d35fac6f5d1d8b7b139e33fc0ffa2,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_0d6d35fac6f5d1d8b7b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T10:43:33.884327"", ""...",2024-01-10T10:43:45.407703,0.9,0.500,0.30,[{'args': {'prompt': 'What are some good habit...,"[{'args': {'source': '(For example, many arti...",[{'args': {'prompt': 'What are some good habit...,11,446,0.000693
4,Sentence Window Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_352dc6fe4f7f114882fe38d1ada7334c,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_352dc6fe4f7f114882f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-10T10:43:46.120694"", ""...",2024-01-10T10:43:58.676656,0.9,0.500,0.20,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Many successful people d...,[{'args': {'prompt': 'How can altruism be bene...,12,547,0.000864


In [44]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.74:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Advanced RAG Pipeline

In [21]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [23]:
from scripts.utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [24]:
from scripts.utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [26]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
pprint(str(window_response))

('To get started on a personal project in AI, it is important to first '
 'identify and scope the project. Consider your career goals and choose a '
 'project that complements them. Ensure that the project is responsible, '
 'ethical, and beneficial to people. As you progress in your career, aim for '
 'projects that grow in scope, complexity, and impact over time. Building a '
 'portfolio of projects that shows skill progression can also be helpful. '
 'Additionally, there are resources available in the provided chapters that '
 'can guide you in starting your AI project and finding the right job in the '
 'field.')


In [35]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [37]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19e962c5650 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19e9cbe2750 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19e962c5650 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19e9cbe2750 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively as a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19e962c5650 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19e9cbe2750 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include keeping up-to-date with evolving technology, finding suitable projects and estimating timelines and return on investment, dealing with the highly iterative nature of AI projects, collaborating with stakeholders who lack expertise in AI, and struggling with technical challenges while reading research papers or tuning neural network hyperparameters.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19e962c5650 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19e9cbe2750 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. The author mentions that they still find many research papers challenging to read and have made mistakes while working with neural networks. They also assure the reader that everyone who has published a seminal AI paper has struggled with similar technical challenges at some point. Therefore, it is clear that finding parts of AI challenging is a common experience.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19e962c5650 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19e9cbe2750 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


What is the right AI job for me?
The right AI job for you can be found by following the steps outlined in Chapter 7: A Simple Framework for Starting Your AI Job Search, Chapter 8: Using Informational Interviews to Find the Right Job, and Chapter 9: Finding the Right AI Job for You. These chapters provide guidance on how to identify and pursue AI job opportunities that align with your career goals and skill progression. Additionally, Chapter 10: Keys to Building a Career in AI offers valuable insights on how to navigate and succeed in the field of AI.


In [45]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Direct Query Engine,0.900000,0.272727,0.800000,14.181818,0.003069
Sentence Window Query Engine,0.890909,0.400000,0.870455,14.181818,0.000847


In [39]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.74:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Auto-merging retrieval

In [46]:
from scripts.utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [47]:
from scripts.utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [48]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 30f6a043-2c7c-4c7f-8256-e4395c97b7cc.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 05b3d21a-3fca-43a4-825c-17e87fa61611.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones. This progression will demonstrate your growth and development over time. Additionally, effective communication is crucial in order to explain your thinking and showcase the value of your work. This will help others see the potential in your projects and trust you with the necessary resources for larger endeavors.


In [49]:
# tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [50]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19ea2c4c950 is calling an instrumented method <function BaseQueryEngine.query at 0x0000019E81E8A8E0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19ea2c4c950 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x19ea2c41cd0 is calling an instrumented method <function BaseRetriever.retrieve at 0x0000019E84873100>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obje

> Merging 2 nodes into parent node.
> Parent node id: c6f849d3-a0ae-4a6b-9d6f-0c3db8fc573c.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 40c24fb8-abaf-401b-9915-9696763e8991.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19ea2c4f610 is calling an instrumented method <function CompactAndRefine.get_response at 0x0000019E848539C0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19ea2c4f610 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job. These steps are supported by being part of a community.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19ea2c4c950 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19ea2c4f610 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


...

Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. The author of the text mentions that even accomplished individuals in the AI community have faced challenges and struggled with technical aspects of AI. They emphasize that everyone has been in a similar position and that it is okay to find certain aspects of AI challenging.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x19ea2c4c950 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000019E8807FD80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x19e887ec650) using this function.


> Merging 1 nodes into parent node.
> Parent node id: 7a29fdec-f47b-4465-863e-d2220a869524.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 0a4480ba-8877-4457-a6a2-373bb5c376c0.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 87171222-2513-4be0-be44-4ffbb70ea26e.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x19ea2c4f610 is calling an instrumented method <function Refine.get_response at 0x0000019E84870EA0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x19e887ec810) using this function.


What is the right AI job for me?
The right AI job for you is discussed in Chapter 9 of the eBook "How to Build a Career in AI."


In [51]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Direct Query Engine,0.900000,0.272727,0.800000,14.181818,0.003069
Sentence Window Query Engine,0.890909,0.400000,0.870455,14.181818,0.000847
Automerging Query Engine,0.845455,0.522727,0.946970,14.181818,0.000850


In [52]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.74:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>